## Scale data

In [1]:
import pandas as pd
import numpy as np
import pickle

def scale_dfs(dfs, start_column_index):
    for var in dfs:
        dfs[var].iloc[:,start_column_index:] = dfs[var].iloc[:,start_column_index:].mul(scales[var])

In [2]:
pm_scale = 1000000000 #to micrograms
o3_scale = 603447629
no2_scale = 629585593
so2_scale = 452118045 #to ppbv

In [3]:
scales = {
    "pm2p5":pm_scale,
    "o3":o3_scale,
    "no2":no2_scale,
    "so2":so2_scale
         }

In [4]:
#pollution_sheet_names = ["pm2p5", "o3", "no2", "so2"]
pollution_sheet_names = ["o3", "no2", "so2"]

main_var_sheet_names = ["maxtempC","mintempC","avgtempC","cloudcover","humidity","precipMM","pressure",
           "windspeedKmph","totalSnow_cm",'sunHour','moon_illumination','moonrise',
           'moonset','sunrise','sunset','DewPointC','FeelsLikeC',#'HeatIndexC',
            'WindChillC','WindGustKmph','visibility','winddirDegree','uvIndex'
          ]

vars_not_to_weight = ['moonrise','moonset','sunrise','sunset']

In [5]:
file_name = "US_pollution_gases"

In [6]:
dfs = pd.read_excel(file_name + ".xlsx", index_col = 0, sheet_name = pollution_sheet_names)

In [7]:
start_column_index = list(dfs["o3"].columns).index("2020-04-01")
start_column_index

459

In [8]:
dfs["o3"].head()

,Location,Lat,Long,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,...,2020-08-22 00:00:00,2020-08-23 00:00:00,2020-08-24 00:00:00,2020-08-25 00:00:00,2020-08-26 00:00:00,2020-08-27 00:00:00,2020-08-28 00:00:00,2020-08-29 00:00:00,2020-08-30 00:00:00,2020-08-31 00:00:00
State,,,,,,,,,,,,,,,,,,,,,
Alabama,"Autauga, Alabama, US",32.539527,-86.644082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alabama,"Baldwin, Alabama, US",30.727750,-87.722071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alabama,"Barbour, Alabama, US",31.868263,-85.387129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alabama,"Bibb, Alabama, US",32.996421,-87.125115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alabama,"Blount, Alabama, US",33.982109,-86.567906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
scale_dfs(dfs, start_column_index)

In [10]:
#dfs["pm2p5"].head()
dfs

OrderedDict([('o3',
                                          Location        Lat        Long  2019-01-01  \
              State                                                                     
              Alabama         Autauga, Alabama, US  32.539527  -86.644082         NaN   
              Alabama         Baldwin, Alabama, US  30.727750  -87.722071         NaN   
              Alabama         Barbour, Alabama, US  31.868263  -85.387129         NaN   
              Alabama            Bibb, Alabama, US  32.996421  -87.125115         NaN   
              Alabama          Blount, Alabama, US  33.982109  -86.567906         NaN   
              Alabama         Bullock, Alabama, US  32.100305  -85.712655         NaN   
              Alabama          Butler, Alabama, US  31.753001  -86.680575         NaN   
              Alabama         Calhoun, Alabama, US  33.774837  -85.826304         NaN   
              Alabama        Chambers, Alabama, US  32.913601  -85.390727         NaN   
 

In [11]:
writer = pd.ExcelWriter('scaled_'+ file_name +'.xlsx', engine='xlsxwriter')
for var in dfs:
    print("Saving", var)
    dfs[var].to_excel(writer, sheet_name = var, index = True)
    
writer.save()

Saving o3
Saving no2
Saving so2


## Replacing Missing Data with avg of neighboring values

In [ ]:
def flag_pollution_value(row, index):
    #After Scaling, threshold chosen after inspection (correct values always more than 10^-3, avg is ~
    return row[index] < 10**(-5)

In [ ]:
def flag_temperature_value(row, index):
    return row[index]==0 and abs(row[index-1]) > 8 and abs(row[index+1]) > 8

In [ ]:
def flag_UV_value(row, index):
    return row[index]==0

In [ ]:
def avg_neighbors(row, index):
    return (row[index-2]+ row[index-1] + row[index+1]+row[index+2])/4

In [ ]:
def interpolate_pollution_data(dfs, start_column_index):
    for var in dfs:
        num_corrections = 0
        arr = dfs[var].to_numpy()
        n = len(arr[0])
        for row in arr:
            for j in range(start_column_index+2,n-2): #avoid first 2 and last 2 vals for avging
                if flag_pollution_value(row, j):
                    row[j] = avg_neighbors(row, j)
                    num_corrections += 1
        dfs[var] = pd.DataFrame(arr, columns = dfs[var].columns, index = dfs[var].index)
        
        print(num_corrections,"corrections made for variable",var)

In [ ]:
def interpolate_temperature_data(dfs, start_column_index):
    num_corrections = 0
    maxTemp = dfs["maxtempC"].to_numpy()
    minTemp = dfs["mintempC"].to_numpy()
    avgTemp = dfs["avgtempC"].to_numpy()
    n = len(maxTemp[0])
    for i, row in enumerate(maxTemp):
        for j in range(start_column_index+2,n-2): #avoid first 2 and last 2 vals for avging
            if flag_temperature_value(row, j):
                row[j] = avg_neighbors(row, j)
                minTemp[i][j] = avg_neighbors(minTemp[i], j)
                avgTemp[i][j] = avg_neighbors(avgTemp[i], j)
                num_corrections+=1
            
    dfs["maxtempC"] = pd.DataFrame(maxTemp, columns = dfs["maxtempC"].columns, index = dfs["maxtempC"].index)
    dfs["mintempC"] = pd.DataFrame(minTemp, columns = dfs["mintempC"].columns, index = dfs["mintempC"].index)
    dfs["avgtempC"] = pd.DataFrame(avgTemp, columns = dfs["avgtempC"].columns, index = dfs["avgtempC"].index)
    
    print(num_corrections,"corrections made for temperature")

In [ ]:
def interpolate_UV_data(dfs, start_column_index):
    num_corrections = 0
    arr = dfs["uvIndex"].to_numpy()
    n = len(arr[0])
    for row in arr:
        for j in range(start_column_index+2,n-2): #avoid first 2 and last 2 vals for avging
            if flag_UV_value(row, j):
                row[j] = avg_neighbors(row, j)
                num_corrections += 1
    dfs["uvIndex"] = pd.DataFrame(arr, columns = dfs["uvIndex"].columns, index = dfs["uvIndex"].index)
        
    print(num_corrections,"corrections made for uvIndex")

In [ ]:
pollution_sheet_names = ["pm2p5", "o3", "no2", "so2"]

main_var_sheet_names = ["maxtempC","mintempC","avgtempC","cloudcover","humidity","precipMM","pressure",
           "windspeedKmph","totalSnow_cm",'sunHour','moon_illumination','moonrise',
           'moonset','sunrise','sunset','DewPointC','FeelsLikeC',#'HeatIndexC',
            'WindChillC','WindGustKmph','visibility','winddirDegree','uvIndex'
          ]

temperature_vars = ["maxtempC","mintempC","avgtempC"]

#### Load Pollution Files Here

In [ ]:
file_name = ""

In [ ]:
dfs = pd.read_excel(file_name + ".xlsx", index_col = 0, sheet_name = pollution_sheet_names)

In [ ]:
start_column_index = list(dfs["pm2p5"].columns).index("2019-01-01")
start_column_index

In [ ]:
interpolate_pollution_data(dfs, start_column_index)

In [ ]:
writer = pd.ExcelWriter('interpolated_'+ file_name +'.xlsx', engine='xlsxwriter')
for var in dfs:
    print("Saving", var)
    dfs[var].to_excel(writer, sheet_name = var, index = True)
    
writer.save()

### Load Temperature Files Here

In [ ]:
file_name = ""

In [ ]:
dfs = pd.read_excel(file_name + ".xlsx", index_col = 0, sheet_name = main_var_sheet_names)

In [ ]:
start_column_index = list(dfs["maxtempC"].columns).index("2019-01-01")
start_column_index

In [ ]:
interpolate_temperature_data(dfs, start_column_index)

In [ ]:
interpolate_UV_data(dfs, start_column_index)

In [ ]:
writer = pd.ExcelWriter('interpolated_'+ file_name +'.xlsx', engine='xlsxwriter')
for var in dfs:
    print("Saving", var)
    dfs[var].to_excel(writer, sheet_name = var, index = True)
    
writer.save()

## Weighted Avg of Weather Variables

Weights by population

In [ ]:
pollution_sheet_names = ["pm2p5", "o3", "no2", "so2"]
main_var_sheet_names = ["maxtempC","mintempC","avgtempC","cloudcover","humidity","precipMM","pressure",
           "windspeedKmph","totalSnow_cm",'sunHour','moon_illumination','moonrise',
           'moonset','sunrise','sunset','DewPointC','FeelsLikeC',#'HeatIndexC',
            'WindChillC','WindGustKmph','visibility','winddirDegree','uvIndex'
          ]

vars_not_to_weight = ['moonrise','moonset','sunrise','sunset']

sheet_names = pollution_sheet_names

In [ ]:
dfs = pd.read_excel("", index_col = 0, 
                    sheet_name = sheet_names)

#dfs = dict: sheet_names --> dataframes

In [ ]:
for var in dfs:
    df = dfs[var]
    df.drop(columns = ["Lat", "Long"], inplace = True)


In [ ]:
start_column_index = list(dfs[sheet_names[0]].columns).index("2019-01-01")
print(start_column_index)
date_columns = list(dfs[sheet_names[0]].columns[start_column_index:])

In [ ]:
countries = list(dict.fromkeys(dfs[sheet_names[0]].index))

In [ ]:
print(countries)

In [ ]:
dfs[sheet_names[0]].loc[countries[0]].iloc[0][:5]

In [ ]:
def weighted_weather_for_country(country, df):
    country_df = df.loc[country]
    weights = np.array(country_df["Population"])
    weights = weights/np.sum(weights)
    row = [country] #output row
    for date in date_columns:
        row.append(np.average(country_df[date], weights = weights))
    
    return row

In [ ]:
output_frames = []
for var in sheet_names:
    data = []
    df = dfs[var]
    for country in countries:
        if var in vars_not_to_weight:
            row = [country] + list(df.iloc[0][start_column_index:])
        else:
            try:
                row = weighted_weather_for_country(country, df)
            except:
                print(country, var)
                break
        data.append(row)
        
    df = pd.DataFrame(data, columns = ["Country"] + date_columns).set_index("Country")
    output_frames.append(df)
    

In [ ]:
writer = pd.ExcelWriter('Weights Applied.xlsx', engine='xlsxwriter') #rename!

In [ ]:
for i, sheet in enumerate(output_frames):
    sheet.to_excel(writer, sheet_name = sheet_names[i], index = True)

In [ ]:
writer.save()

## Reorder Global Rows based on canonical ordering

In [ ]:
import pickle
import pandas as pd

In [ ]:
canonical_order = pickle.load(open("canonical_order.p", "rb"))

In [ ]:
canonical_order

In [ ]:
weighted_countries = pickle.load(open("weighted_countries_list.p", "rb"))

In [ ]:
weighted_countries

In [ ]:
pollution_sheet_names = ["pm2p5", "o3", "no2", "so2"]

main_var_sheet_names = ["maxtempC","mintempC","avgtempC","cloudcover","humidity","precipMM","pressure",
           "windspeedKmph","totalSnow_cm",'sunHour','moon_illumination','moonrise',
           'moonset','sunrise','sunset','DewPointC','FeelsLikeC',#'HeatIndexC',
            'WindChillC','WindGustKmph','visibility','winddirDegree','uvIndex'
          ]

In [ ]:
weighted_filename = ""
simple_filename = ""

In [ ]:
weighted_dfs = pd.read_excel(weighted_filename + ".xlsx", index_col = 0, sheet_name = pollution_sheet_names)

In [ ]:
simple_dfs = pd.read_excel(simple_filename + ".xlsx", index_col = 0, sheet_name = pollution_sheet_names)

In [ ]:
simple_dfs["pm2p5"]

In [ ]:
for var in simple_dfs:
    simple_dfs[var].drop(weighted_countries, inplace = True)

In [ ]:
final_dfs = {var:simple_dfs[var].append(weighted_dfs[var]) for var in simple_dfs}

In [ ]:
columns = ["lat","long"] + list(weighted_dfs["pm2p5"].columns)

In [ ]:
columns

In [ ]:
for var in final_dfs:
    final_dfs[var] = final_dfs[var].reindex(canonical_order)
    final_dfs[var] = final_dfs[var][columns]

In [ ]:
final_dfs["pm2p5"]

In [ ]:
final_dfs.keys()

In [ ]:
writer = pd.ExcelWriter('Merged Global Pollution.xlsx', engine='xlsxwriter')

In [ ]:
for var in final_dfs:
    final_dfs[var].to_excel(writer, sheet_name = var, index = True)

In [ ]:
writer.save()